In [140]:
import backtester as bt
import numpy as np
import datetime
import yfinance as yf
import matplotlib.pyplot as plt

In [200]:
ticker = []
ticker.append("AAPL")
ticker.append("TSLA")
ticker.append("MSFT")
ticker.append("GOOG")

In [203]:
start = datetime.datetime(2021, 9, 16)
end = datetime.datetime(2022, 9, 18)
data = yf.download(tickers = ticker, start=start, end=end, interval = "1d")
data = data['Adj Close']
size = len(ticker)

[*********************100%***********************]  4 of 4 completed


In [247]:
def min_variance(data):

    #Convert data into daily returns, average daily returns and annual returns
    size = data.shape[1]
    data_returns = data.pct_change()
    data_returns = data_returns[1:]
    daily_average = np.mean(data_returns,axis = 0)
    annual_average = ((daily_average+1)**365)-1
    #Construct Covariance matrix
    cov_data = np.cov(data_returns.T)
    #Construct other vectors
    bottom = np.ones(size)
    bottom2 = np.ones(size+1)
    bottom2[-1] = 0
    beta = np.zeros(size+1)
    beta[-1] = 1
    #Concatenate all Vectors with Covariance Matrix
    test = np.c_[2*cov_data,bottom.T]
    final = np.row_stack((test, bottom2))
    answer = np.matmul(np.linalg.inv(final),beta)

    #Compute NP Arrays that contain:
    #Portfolio Weights
    weights = np.around(answer,3)
    p_weights = weights[0:-1]
    weights1 = np.array(p_weights, dtype=float)
    #Portfolio Mean
    p_mean = np.dot(weights1,annual_average)
    #Portfolio Variance
    p_variance = np.matmul(np.matmul(weights1.T,cov_data),weights1)
    return p_weights, p_mean, p_variance


In [248]:
def min_variance_mean(data, target_mean):
    #Set Target Mean
    mean = target_mean
    size = data.shape[1]
    #Convert data into daily returns, average daily returns and annual returns
    data_returns = data.pct_change()
    data_returns = np.array(data_returns[1:])
    daily_average = np.mean(data_returns,axis = 0)
    annual_average = ((daily_average+1)**365)-1
    #Construct Covariance matrix
    cov_data = np.cov(data_returns.T)
    #Construct other vectors
    bottom = np.ones(size)
    bottom2 = np.ones(size+2)
    bottom2[-2] = 0
    bottom2[-1] = 0
    beta = np.zeros(size+2)
    beta[-2] = mean
    beta[-1] = 1
    #Concatenate all Vectors with Covariance Matrix
    test = np.c_[2*cov_data,annual_average.T,bottom.T]
    annual_average = np.append(annual_average,np.array([0,0]))
    final = np.row_stack((test,annual_average, bottom2))
    #Compute weights
    answer = np.matmul(np.linalg.inv(final),beta)
    #Round weights
    weights = np.around(answer,3)
    p_weights = weights[0:-2]

    #Compute NP Arrays that contain:
    #Portfolio Weights
    weights1 = np.array(p_weights, dtype=float)
    #Portfolio Mean
    p_mean = np.dot(weights1,annual_average[0:size])
    #Portfolio Variance
    p_variance = np.matmul(np.matmul(weights1.T,cov_data),weights1)
    return p_mean, p_variance, p_weights

In [249]:
def Efficient_Frontier(data,max_mean):
    means = np.linspace(0.01,max_mean,10)
    mean_value = []
    variance_value = []
    for i in means:
        #Obtain portfolio mean and variance
        p_mean, p_variance, p_weights= min_variance_mean(data,i)
        #Store each value in list
        mean_value.append(p_mean)
        variance_value.append(p_variance)
    #Plot the Efficient Frontier
    plt.scatter(variance_value,mean_value)
    plt.xlabel('Portfolio Variance (%)')
    plt.ylabel('Portfolio Return (%)')
    plt.title("Efficient Frontier")
    plt.show()

In [251]:
p_mean, p_variance, p_weights = min_variance_mean(data,0.5)
print(p_mean)
print(p_variance)
print(p_weights)

0.4996899074734121
0.000636515127842687
[ 1.426 -0.602 -0.026  0.201]
